## NASA Center Analysis
<details>
<summary>VARIABLES</summary>

| Variable Name       | Long Name                                          | Variable Category | Units     | Description                                                                                                                                                          |
| ------------------- | -------------------------------------------------- | ----------------- | --------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| tmax_days_35C       | \# days Tmax ≥35°C                                 | extreme index     | \# days   | Number of days, per year, with Tmax >=35C                                                                                                                            |
| tmax_days_90th      | \# days Tmax ≥90th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 90th percentile. 90th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_95th      | \# days Tmax ≥95th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 95th percentile. 95th percentile calculated using all daily tmax values from 1995-2014.                                          |
| tmax_days_99th      | \# days Tmax ≥99th Percentile                      | extreme index     | \# days   | Number of days, per year, with Tmax 99th percentile. 99th percentile calculated using all daily tmax values from 1995-2014.                                          |
| Hottest_Tmax        | Hottest Tmax of the Year (°C)                      | extreme index     | degrees C | Hottest Tmax value every year                                                                                                                                        |
| Max_DTR             | Largest Diurnal Temperature Range of the Year (°C) | extreme index     | degrees C | largest diurnal temperature range (tmax minus tmin) each year                                                                                                        |
| tmin_tropnights_20C | \# days Tmin ≥20°C                                 | extreme index     | \# days   | Number of days, per year with Tmin >=20C                                                                                                                             |
| tmin_frostdays_0C   | \# days Tmin ≤0°C                                  | extreme index     | \# days   | Number of days per year with Tmin <=0C                                                                                                                               |
| Coldest_Tmin        | Coldest Tmin of the Year (°C)                      | extreme index     | degrees C | Coldest minimum temperature each year                                                                                                                                |
| prec_days_dry       | \# days with precipitation ≤0.001 in               | extreme index     | \# days   | Number of days, per year, where precipitation <=1e-3 inches                                                                                                          |
| prec_days_oneinch   | \# days with precipitation ≥1 in                   | extreme index     | \# days   | Number of days, per year, where precipitation >=1 inch                                                                                                               |
| prec_days_90th      | \# days with precipitation ≥90th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=90th percentile. 90th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_95th      | \# days with precipitation ≥95th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=95th percentile. 95th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| prec_days_99th      | \# days with precipitation ≥99th Percentile        | extreme index     | \# days   | Number of days, per year, where precipitation >=99th percentile. 99th percentile calculated usingd all daily precipitation values (dry days EXCLUDED) from 1995-2014 |
| tmax_annave         | Annual Average Tmax (°C)                           | annual average    | degrees C | Annual average maximum daily temperature                                                                                                                             |
| tmin_annave         | Annual Average Tmin (°C)                           | annual average    | degrees C | Annual average minimum daily temperature                                                                                                                             |
| prec_annave         | Annual Total Precipitation (mm)                    | annual SUM        | degrees C | Annual SUM of precipitation                                                                                                                                          |</details>

In [1]:
# Imports
import os
import warnings
import zipfile
import numpy as np
import pandas as pd
import pandasql as psql

# Suppress warnings
warnings.filterwarnings('ignore')

## Initialization

In [2]:
path = 'updated_extremes'  # data directory
center = 'LARC'.upper()    # NASA center to analyze
path = os.path.join(path, center)
only_future = True         # flag to use only 2020-2099
ssp = ['ssp126', 'ssp245', 'ssp370'] # scenarios to use

In [3]:
# DO NOT CHANGE THIS CELL
# File name convention: variable.ssp###.CENTER.csv

# NASA Centers
centers = ['AMES', 'GSFC', 'JPL', 'KSC', 'MSFC', 'MAF', 'GISS',
           'LARC', 'SSC', 'GRC', 'WFF', 'JSC', 'WSTF', 'AFRC']

# Check if the provided center is valid
if center not in centers:
    raise ValueError(f'{center} not in {centers}')

# Variable unit: number of DAYS when... assume others are celsius
day_unit = ['days', 'tropnights']

# Shared Socioeconomic Pathways (SSPs) climate scenarios
print(f'Available: {sorted(list({f.split('.')[1] for f in os.listdir(path) 
                                 if 'ssp' in f.split('_')[-1]}))}')
# Time periods: 10 years before+after a decade
time_periods = {'short': (2020, 2049),  # 2030's: 2020-2029, 2030-2039, 2040-2049
                'mid':   (2040, 2069),  # 2050's: 2040-2049, 2050-2059, 2060-2069
                'long':  (2070, 2099),  # 2080's: 2070-2079, 2080-2089, 2090-2099
                }

Available: ['ssp126', 'ssp245', 'ssp370']


# Get Files/Data

In [4]:
def get_files(path: str, center: str):
    '''Returns list of all csv files in the directory that contain the center and ssp name'''
    return [os.path.join(path, f) for f in os.listdir(path) 
             if center in f and any(s in f for s in ssp) and f.endswith('.csv')]

def check_df_consistency(df_list: list[pd.DataFrame]):
    '''Returns T/F if all dataframes in the list have the same column names and index values'''
    if not df_list:
        return False
    
    # Get reference column names and index values from the first dataframe
    ref_cols, ref_index = list(df_list[0].columns), list(df_list[0].index)
    
    # Check if all other dataframes have the same column names and index values
    return all(list(df.columns) == ref_cols and list(df.index) == ref_index 
               for df in df_list[1:])

def label_term(year: int):
    '''Returns list of time period labels for a given year'''
    return [t for t, (s, e) in time_periods.items() if s <= year <= e]


def preprocess(filename: str, only_future: bool=True):
    '''Returns a preprocessed pandas DataFrame from a csv file'''
    df = pd.read_csv(filename)
    name = filename.split('/')[-1][:-4].split('.')
    
    # Add new columns: term, scenario, and variable
    df.insert(0, 'term', df.Years.apply(label_term))
    df.insert(0, 'scenario', name[1])
    df.insert(0, 'variable', name[0] + ('_d' if any(d in filename for d in day_unit) else '_c'))
    
    # Explode the 'term' column (in case a year belongs to multiple terms)
    df = df.explode('term')

    # Remove rows with NaN terms if only_future is True, otherwise return all rows
    return df.dropna(subset=['term']) if only_future else df # nan's (unlabeled) assumed to be past data

In [5]:
# Preprocess all files as df's
files = sorted(get_files(path, center))
df = [preprocess(f, only_future) for f in files]

# Check if all dataframes have the same format
if not check_df_consistency(df):
    raise ValueError('DataFrames are inconsistent')

# Combine all dataframes into one
df = pd.concat(df).reset_index(drop=True)
cols = ['variable', 'scenario', 'term']

# Check the number of years per time period (expected is 30)
years_per_term = df.groupby(cols).size().unique()
if len(years_per_term) != 1 or years_per_term[0] != 30:
    raise ValueError(
        f'# of years per time period is incorrect: {years_per_term}')

print(f'{len(files)} {center} files')
print(files[:5], '\n')
display(df.info())

51 LARC files
['updated_extremes/LARC/Coldest_Tmin.ssp126.LARC.csv', 'updated_extremes/LARC/Coldest_Tmin.ssp245.LARC.csv', 'updated_extremes/LARC/Coldest_Tmin.ssp370.LARC.csv', 'updated_extremes/LARC/Hottest_Tmax.ssp126.LARC.csv', 'updated_extremes/LARC/Hottest_Tmax.ssp245.LARC.csv'] 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4590 entries, 0 to 4589
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   variable       4590 non-null   object 
 1   scenario       4590 non-null   object 
 2   term           4590 non-null   object 
 3   Years          4590 non-null   float64
 4   ACCESS-CM2     4590 non-null   float64
 5   ACCESS-ESM1-5  4590 non-null   float64
 6   BCC-CSM2-MR    4590 non-null   float64
 7   CESM2          1620 non-null   float64
 8   CMCC-ESM2      4590 non-null   float64
 9   CNRM-CM6-1     4590 non-null   float64
 10  CNRM-ESM2-1    4590 non-null   float64
 11  EC-Earth3      4590 non-null 

None

# Quality Check

In [6]:
df.groupby('variable').max()

,scenario,term,Years,ACCESS-CM2,ACCESS-ESM1-5,BCC-CSM2-MR,CESM2,CMCC-ESM2,CNRM-CM6-1,CNRM-ESM2-1,...,MPI-ESM1-2-LR,MRI-ESM2-0,NorESM2-LM,NorESM2-MM,MME-mean,MME-median,MME-pct25,MME-pct75,MME-pct05,MME-pct95
variable,,,,,,,,,,,,,,,,,,,,,
Coldest_Tmin_c,ssp370,short,2099.0,-0.931519,-3.823822,-2.503448,NaN,-2.908722,-3.291687,-4.559418,...,-3.282806,-1.653656,-3.980804,-3.331390,-6.561266,-6.352448,-7.718903,-4.259308,-10.060699,-1.363556
Hottest_Tmax_c,ssp370,short,2099.0,47.414276,45.155914,47.664703,NaN,45.756561,46.238922,47.965546,...,43.820068,45.541718,44.691162,47.794220,42.553859,42.776978,41.717285,43.820068,39.927338,47.414276
Max_DTR_c,ssp370,short,2099.0,34.059143,29.701355,39.230255,NaN,31.971222,37.473969,33.203094,...,29.767059,38.054535,28.248413,32.301270,26.489332,27.029785,24.990265,29.414246,23.054901,37.861099
annmean_tmax_c,ssp370,short,2099.0,27.312395,26.693493,27.229473,NaN,26.427290,27.777250,26.785118,...,25.850698,25.449072,25.747143,27.040831,25.558043,25.747143,25.033554,26.176758,24.429266,27.306044
annmean_tmin_c,ssp370,short,2099.0,15.580885,14.985955,15.211082,NaN,15.626901,16.496094,15.355649,...,15.402845,14.750004,14.672589,15.639556,14.736029,14.672589,14.199707,15.153758,13.651192,16.911816
annsum_prec_c,ssp370,short,2099.0,1685.417893,1801.064932,1786.310981,1808.596818,1860.659329,1643.072450,1657.399828,...,1816.360349,1836.667256,1742.415865,1749.636610,1426.768415,1447.683173,1304.761080,1579.149605,1218.777148,1775.081698
prec_days_90th_d,ssp370,short,2099.0,42.000000,48.000000,40.000000,41.000000,39.000000,35.000000,39.000000,...,57.000000,41.000000,47.000000,42.000000,29.500000,30.500000,27.250000,35.000000,22.050000,47.650000
prec_days_95th_d,ssp370,short,2099.0,28.000000,27.000000,22.000000,25.000000,29.000000,21.000000,25.000000,...,32.000000,27.000000,30.000000,22.000000,18.000000,18.500000,14.000000,21.750000,11.000000,28.700000
prec_days_99th_d,ssp370,short,2099.0,9.000000,12.000000,10.000000,8.000000,9.000000,8.000000,9.000000,...,13.000000,9.000000,10.000000,7.000000,5.500000,5.000000,3.250000,7.500000,2.050000,12.000000


In [7]:
df.groupby('variable').min()

,scenario,term,Years,ACCESS-CM2,ACCESS-ESM1-5,BCC-CSM2-MR,CESM2,CMCC-ESM2,CNRM-CM6-1,CNRM-ESM2-1,...,MPI-ESM1-2-LR,MRI-ESM2-0,NorESM2-LM,NorESM2-MM,MME-mean,MME-median,MME-pct25,MME-pct75,MME-pct05,MME-pct95
variable,,,,,,,,,,,,,,,,,,,,,
Coldest_Tmin_c,ssp126,long,2020.0,-20.428391,-20.410950,-30.617416,NaN,-19.131744,-24.557144,-22.943710,...,-19.054031,-24.631653,-15.746368,-21.732437,-13.303314,-13.215698,-15.710175,-10.944702,-23.594238,-9.756409
Hottest_Tmax_c,ssp126,long,2020.0,36.378784,34.746796,36.579468,NaN,35.086365,34.750946,35.739655,...,34.268738,36.197693,34.381500,36.164886,37.705517,37.221222,35.866425,38.633484,33.838257,39.813721
Max_DTR_c,ssp126,long,2020.0,19.484528,20.472870,20.711517,NaN,19.935699,20.811005,19.885498,...,19.211548,20.189728,18.386230,20.936829,23.059900,22.423767,21.014496,23.827087,19.194702,25.837036
annmean_tmax_c,ssp126,long,2020.0,21.218201,21.424860,20.746437,NaN,19.891321,21.214272,21.130535,...,20.588789,21.272921,21.530184,21.282082,22.055873,21.920818,21.448803,22.300543,20.588789,22.887661
annmean_tmin_c,ssp126,long,2020.0,9.779553,9.857891,9.466441,NaN,9.082039,9.659132,10.215306,...,9.791513,9.776453,9.917004,10.140264,10.833823,10.774098,10.347320,11.001023,9.539014,11.684646
annsum_prec_c,ssp126,long,2020.0,915.268630,836.970968,865.837182,886.217967,990.703695,771.679358,843.144565,...,876.891344,872.979374,758.049076,776.496225,1153.638787,1128.614293,1028.393563,1234.365352,885.312174,1307.057740
prec_days_90th_d,ssp126,long,2020.0,16.000000,16.000000,11.000000,11.000000,10.000000,9.000000,12.000000,...,12.000000,13.000000,13.000000,8.000000,22.272727,20.500000,16.500000,24.750000,11.200000,29.000000
prec_days_95th_d,ssp126,long,2020.0,5.000000,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,...,7.000000,5.000000,6.000000,2.000000,10.863636,10.500000,7.250000,13.000000,3.250000,15.000000
prec_days_99th_d,ssp126,long,2020.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,1.000000,3.000000,0.000000,4.000000


In [8]:
MME = list(df.filter(regex='MME-').columns)
models = list(df.columns.drop(cols+MME+['Years']))
print(models)
df1 = df.set_index(['variable', 'scenario', 'term', 'Years'])
df1['calc_mean'] = df1[models].mean(axis=1)
df1['calc_median'] = df1[models].median(axis=1)
df1['calc_pct25'] = df1[models].quantile(0.25, axis=1)
df1['calc_pct75'] = df1[models].quantile(0.75, axis=1)
df1['calc_pct05'] = df1[models].quantile(0.05, axis=1)
df1['calc_pct95'] = df1[models].quantile(0.95, axis=1)

df_mme = df1.filter(regex='MME-').round(10)
df_cal = df1.filter(regex='calc_').round(10)
df_cal.columns=df_mme.columns

err = df_mme.compare(df_cal, result_names=('mme', 'recalc')).dropna(how='all')
display(err)
display(err.info())

['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CESM2', 'CMCC-ESM2', 'CNRM-CM6-1', 'CNRM-ESM2-1', 'EC-Earth3', 'FGOALS-g3', 'GFDL-ESM4', 'GISS-E2-1-G', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'KACE-1-0-G', 'MIROC6', 'MIROC-ES2L', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NorESM2-LM', 'NorESM2-MM']


MME-pct05             
                                             mme       recalc
variable      scenario term  Years                           
annsum_prec_c ssp370   short 2042.0  1012.398082  1012.398082
                       mid   2042.0  1012.398082  1012.398082

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2 entries, ('annsum_prec_c', 'ssp370', 'short', np.float64(2042.0)) to ('annsum_prec_c', 'ssp370', 'mid', np.float64(2042.0))
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   (MME-pct05, mme)     2 non-null      float64
 1   (MME-pct05, recalc)  2 non-null      float64
dtypes: float64(2)
memory usage: 3.8+ KB


None

# Calculations

In [9]:
# Extract column names
MME = list(df.filter(regex='MME-').columns)
models = list(df.columns.drop(cols+MME+['Years']))

# Calculate term-wise statistics
# For variables ending with '_d', use median
# For variables ending with '_c', use mean
term_MME = pd.concat([df[df.variable.str.endswith('_d')].groupby(cols)[MME].median(),
                      df[df.variable.str.endswith('_c')].groupby(cols)[MME].mean()
                     ]).reset_index().sort_values(cols, ascending=[1, 1, 0],
                                                  ignore_index=True)
display(term_MME.head())

,variable,scenario,term,MME-mean,MME-median,MME-pct25,MME-pct75,MME-pct05,MME-pct95
0,Coldest_Tmin_c,ssp126,short,-11.311137,-11.074007,-13.224982,-8.789090,-16.869729,-6.812506
1,Coldest_Tmin_c,ssp126,mid,-10.754145,-10.270861,-12.761407,-8.197082,-16.595351,-6.073173
2,Coldest_Tmin_c,ssp126,long,-10.463190,-9.858391,-12.552755,-7.734112,-16.332220,-5.732386
3,Coldest_Tmin_c,ssp245,short,-11.276643,-10.740965,-13.566644,-8.679038,-17.075702,-6.521618
4,Coldest_Tmin_c,ssp245,mid,-10.279670,-9.937641,-12.157329,-7.821688,-15.826719,-5.600627


## Calculate Change Per (variable, scenario)
- short - mid
- short - long
- mid - long

In [10]:
# query = """
# SELECT
#     a.variable,
#     a.scenario,
#     CASE
#         WHEN a.term = 'short' AND b.term = 'mid' THEN 'short-mid'
#         WHEN a.term = 'short' AND b.term = 'long' THEN 'short-long'
#         WHEN a.term = 'mid' AND b.term = 'long' THEN 'mid-long'
#     END AS term_diff,
#     b.'MME-mean' - a.'MME-mean' AS 'MME-mean',
#     b.'MME-median' - a.'MME-median' AS 'MME-median',
#     b.'MME-pct25' - a.'MME-pct25' AS 'MME-pct25',
#     b.'MME-pct75' - a.'MME-pct75' AS 'MME-pct75'
# FROM term_MME a
# JOIN term_MME b
#     ON a.variable = b.variable
#     AND a.scenario = b.scenario
#     AND (
#         (a.term = 'short' AND b.term = 'mid') OR
#         (a.term = 'short' AND b.term = 'long') OR
#         (a.term = 'mid' AND b.term = 'long')
#     )
# ORDER BY 1, 2, 3 DESC
# """

# change = psql.sqldf(query, locals())

# display(change.head(2))

# Results
## Degree C

In [11]:
# Degrees C
agg_c = (term_MME[term_MME.variable.str.endswith('_c')]
         .groupby(['variable', 'term'])
         .agg({'MME-mean': ['min', 'max']})
         .sort_values(['variable', 'term'], ascending=[1, 0]))

agg_c['rounded_min'], agg_c['rounded_max'] = agg_c['MME-mean'].round(1).values.T
agg_c['diff'] = (agg_c[('MME-mean', 'max')] - agg_c[('MME-mean', 'min')]).round(1)

agg_c.astype(int)


MME-mean       rounded_min rounded_max diff
                          min   max                             
variable       term                                             
Coldest_Tmin_c short      -11   -11         -11         -11    0
               mid        -10   -10         -10         -10    0
               long       -10    -7         -10          -7    2
Hottest_Tmax_c short       38    39          38          39    0
               mid         39    39          39          39    0
               long        39    41          39          41    2
Max_DTR_c      short       24    24          24          24    0
               mid         24    24          24          24    0
               long        24    24          24          24    0
annmean_tmax_c short       22    22          22          22    0
               mid         23    23          23          23    0
               long        23    24          23          24    1
annmean_tmin_c short       11    11          11          11    0
               mid         11    12          11          12    0
               long        11    13          11          13    1
annsum_prec_c  short     1246  1261        1246        1261   15
               mid       1269  1273        1269        1273    4
               long      1284  1323        1284        1323   39

## Days

In [12]:
# Days
agg_d = (term_MME[term_MME.variable.str.endswith('_d')]
          .groupby(['variable', 'term'])
          .agg({'MME-median': ['min', 'max']})
          .sort_values(by=['variable', 'term'], ascending=[1, 0]))

round_up_half = lambda x: np.ceil(x) if x % 1 == 0.5 else round(x)

agg_d['rounded_min'] = agg_d[('MME-median', 'min')].apply(round_up_half).astype(int)
agg_d['rounded_max'] = agg_d[('MME-median', 'max')].apply(round_up_half).astype(int)
agg_d['diff'] = (agg_d['rounded_max'] - agg_d['rounded_min']).astype(int)

agg_d

MME-median         rounded_min rounded_max diff
                                   min     max                             
variable              term                                                 
prec_days_90th_d      short      24.75   25.50          25          26    1
                      mid        26.00   26.00          26          26    0
                      long       26.00   27.50          26          28    2
prec_days_95th_d      short      13.00   13.00          13          13    0
                      mid        13.25   13.50          13          14    1
                      long       13.00   14.75          13          15    2
prec_days_99th_d      short       3.00    3.00           3           3    0
                      mid         3.00    3.00           3           3    0
                      long        3.00    4.00           3           4    1
prec_days_dry_d       short     119.25  122.25         119         122    3
                      mid       117.25  121.25         117         121    4
                      long      119.75  122.50         120         123    3
prec_days_oneinch_d   short       6.00    6.00           6           6    0
                      mid         6.00    7.00           6           7    1
                      long        6.50    7.75           7           8    1
tmax_days_35C_d       short      14.00   15.00          14          15    1
                      mid        16.00   21.50          16          22    6
                      long       16.00   36.00          16          36   20
tmax_days_90th_d      short      49.00   52.00          49          52    3
                      mid        55.00   66.00          55          66   11
                      long       56.50   90.00          57          90   33
tmax_days_95th_d      short      26.50   28.00          27          28    1
                      mid        29.00   35.00          29          35    6
                      long       29.00   60.00          29          60   31
tmax_days_99th_d      short       5.00    5.50           5           6    1
                      mid         6.00    8.00           6           8    2
                      long        6.00   18.00           6          18   12
tmin_frostdays_0C_d   short      44.00   46.00          44          46    2
                      mid        39.00   42.00          39          42    3
                      long       29.00   43.00          29          43   14
tmin_tropnights_20C_d short      85.00   86.00          85          86    1
                      mid        88.50   98.50          89          99   10
                      long       89.50  119.00          90         119   29